## 유튜브소스코드 추출
- beautifulsoup 한계로 selenium사용

### 2023-02-22
song_df 데이터 프레임에 정리된 곡명 가수명을 쿼리값으로 유튜브 검색후 첫번째 영상을 추출시
광고, 검색값과 다른 영상, 검색값이 없는경우 등 오류가 있음
해결해보기위해 유튜브 api사용해보기로 함

In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time # 딜레이 기능 - sleep(초)

# 판다스
import pandas as pd

In [2]:
song_df = pd.read_excel('./music.xlsx') 

In [3]:
# 곡명-가수명으로 검색한 url 리스트 만들기
url_lst = []
for i in range(len(song_df)):
    song = song_df.iloc[i][0] # 곡명담기
    singer = song_df.iloc[i][1] # 가수명담기
    url = f'https://www.youtube.com/results?search_query={singer}+-+{song}+lyrics' # 가수명-곡명 lyrics으로 검색한 url
    url_lst.append(url) # url리스트에 담아주기

### 어떻게 필요한 영상만 담을까??
- 첫번째 for문 -> 모든 곡에대한 url 접근
- 두번째 for문 -> 한음원 url에 접근후 나온페이지의 모든 영상에 접근
- if문 -> 노래명 가수명이 포함일시 링크따오고 break, 추가적으로 a, b, c 등해서 어떤 조건에서 따온 url인지도 판단하는 리스트 컬럼 만들어보기
- elif -> 가사 또는 lyrics이 제목에 포함일시 링크따오고 break
- 해당 url에 원하는 영상 링크를 못따올시 처리방법 생각

In [4]:
# 영상url 담아줄 리스트
yt_lst = []

# 어떤 조건에서 url이 담겨졌는지 체크할 리스트
yt_lst_check = []

# 검색할 url_lst 만큼 반복
for i in range(len(url_lst)):
    driver = wb.Chrome()
    driver.get(url_lst[i])
    time.sleep(1)
    
    # 검색하여 만들어진 유튜브 웹페이지에서 영상제목의 링크가 담긴 a태그, text가 담긴 태그에 접근
    aTag = driver.find_elements(By.CSS_SELECTOR, value = 'ytd-video-renderer #video-title')
    name = driver.find_elements(By.CSS_SELECTOR, value = 'ytd-video-renderer yt-formatted-string')
    
    # 접근한 a태그에서 링크 추출하기
    if len(aTag)>0: # 웹페이제 접근했을때 검색 결과가 있을시
        for j in range(len(aTag)):
            
            # 첫번째 조건 - 접근할 영상 제목에 곡명 가수명 일치할시에 aTag에 접근해서 링크 추출
            if (song_df.iloc[0][0]) in name[j].text and (song_df.iloc[0][1]) in name[j].text:
                yt = aTag[j].get_attribute('href') # a태그의 href속성값 추출
                yt_lst.append(yt) # 추출한 href속성값 yt_lst에 담아주기
                yt_lst_check.append("a") # 곡명 가수명 포함시
                break
                
            # 두번째 조건 영상 제목에 가사 또는 lyrics가 포함되어있을시 aTag 접근해서 링크  추출
            elif '가사' in name[j].text or 'lyrics' in name[j].text:
                yt = aTag[j].get_attribute('href') # a태그의 href속성값 추출
                yt_lst.append(yt) # 추출한 href속성값 yt_lst에 담아주기
                yt_lst_check.append("b") # 가수 or lyrics 가 포함시
                break
                
    else: # 웹페이지에 접근했을때 검색 결과가 없을시
            yt_lst.append("검색결과없음") 
            yt_lst_check.append("c") # 검색결과 없을시
driver.quit()

In [19]:
dic = {'유튜브':yt_lst, '체크':yt_lst_check}

In [20]:
yt_df = pd.DataFrame(dic)

In [21]:
yt_df

,유튜브,체크
0,https://www.youtube.com/watch?v=evOsUf9en-Y,a
1,https://www.youtube.com/watch?v=zVTO8br-6XQ,b
2,https://www.youtube.com/watch?v=hjfeLAGT47s,b
3,https://www.youtube.com/watch?v=JEJGD9mIYQQ,b
4,https://www.youtube.com/watch?v=BEdrr7Je1bU&pp...,b
...,...,...
738,https://www.youtube.com/watch?v=sA7OcLYZ2qk,b
739,https://www.youtube.com/watch?v=8tem27lgr1Q,b
740,https://www.youtube.com/watch?v=Q3eh6pxYAJs,b
741,https://www.youtube.com/watch?v=2AKh2T9FTQA,b


In [22]:
yt_df.to_excel('./yt.xlsx', index = False)

In [5]:
song_df['유튜브'] = yt_lst
song_df['유튜브링크체크'] = yt_lst_check
song_df

ValueError: Length of values (743) does not match length of index (800)